# Suicide analysis
### Lukas Forst

In [76]:
import pandas as pd
import numpy as np
import plotly as plt
import seaborn as sns

In [78]:
# fix plotly not showing graphs sometimes
plt.offline.init_notebook_mode(connected=True)

### Data insight

Now let's look at the data.

In [341]:
data = pd.read_csv('san_data.csv')
data.sample(5)

,Unnamed: 0,country,year,sex,age,suicides_no,population,gdp_for_year,gdp_per_capita,generation,continent
16705,16706,Mexico,2005,Male,25-34,956,9355305.0,8.774762e+11,9071,Generation X,Americas
17130,17131,Netherlands,1999,Male,15-24,82,960000.0,4.419753e+11,29799,Generation X,Europe
20846,20847,Russian Federation,1993,Male,55-74,10118,10681300.0,4.350837e+11,3160,Silent,Europe
9390,9391,France,2014,Female,35-54,778,8608404.0,2.852166e+12,47318,Generation X,Europe
13611,13612,Japan,2012,Male,25-34,2244,7384000.0,6.203213e+12,51379,Millenials,Asia


We must polish the data since the `id` or as it is interpreted `Unnamed: 0` is useless.

In [342]:
data.drop(data.columns[0], axis=1,inplace=True) 
data.sample(5)

,country,year,sex,age,suicides_no,population,gdp_for_year,gdp_per_capita,generation,continent
6903,Cyprus,2000,Female,25-34,0,71455.0,1.018332e+10,11635,Generation X,Asia
16634,Mexico,1999,Male,25-34,726,8214958.0,6.002329e+11,6800,Generation X,Americas
15939,Malta,2003,Male,35-54,4,57171.0,5.456584e+09,14444,Boomers,Europe
7326,Czech Republic,2012,Female,55-74,106,1333013.0,2.073764e+11,20898,Boomers,Europe
1134,Armenia,2006,Female,75+,2,78684.0,6.384452e+09,2310,Silent,Asia


In [338]:
data.describe()

,year,suicides_no,population,gdp_for_year,gdp_per_capita
count,27660.000000,27660.000000,2.766000e+04,2.766000e+04,27660.000000
mean,2001.173102,243.413485,1.850689e+06,4.471009e+11,16815.568330
std,8.418754,904.490185,3.920658e+06,1.457574e+12,18861.585497
min,1985.000000,0.000000,2.780000e+02,4.691962e+07,251.000000
25%,1994.000000,3.000000,9.753525e+04,8.976208e+09,3436.000000
50%,2002.000000,25.000000,4.307250e+05,4.801361e+10,9283.000000
75%,2008.000000,132.000000,1.491041e+06,2.602024e+11,24796.000000
max,2015.000000,22338.000000,4.380521e+07,1.812071e+13,126352.000000


In [330]:
def group_yearly(df, subset = ['year', 'suicides_no', 'population'], agg = {'population':'sum', 'suicides_no':'sum'}):
    '''
    Takes data frame, create subselect by given subset, groups by year and use aggregation function with given dict.abs
    '''
    return df[subset].groupby('year').agg(agg)

Let's visualise yearly values.

In [331]:
yearly = group_yearly(data)
yearly.describe()

,population,suicides_no
count,3.100000e+01,31.000000
mean,1.651292e+09,217187.645161
std,2.917111e+08,42969.612753
min,1.008600e+09,116063.000000
25%,1.539583e+09,207556.500000
50%,1.745247e+09,233408.000000
75%,1.850237e+09,243515.500000
max,1.997297e+09,256119.000000


In [306]:
years = yearly.index.values

Import the graphic tools.

In [307]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

And plot the yearly data.

In [308]:
fig = fig = make_subplots(specs=[[{"secondary_y": True}]])

# add summed suicides
fig.add_trace(
    go.Scatter(x = years, y = yearly['suicides_no'], mode = 'lines', name = 'Suicides'),
    secondary_y=False
)
# add populations
fig.add_trace(
    go.Scatter(x = years, y = yearly['population'], mode = 'lines', name = 'Population'),
    secondary_y=True
)

fig.update_layout(title_text="Suicides and population")

fig.update_xaxes(title_text="years")

fig.update_yaxes(title_text="suicides count", secondary_y=False)
fig.update_yaxes(title_text="population", secondary_y=True)

fig.show()

Let's see some normalized data - suicides per 100 000 people, to see the trends.

In [311]:
def get_suicides_per_100k(df):
    return(100_000 * df['suicides_no']) / df['population']

In [312]:
yearly = group_yearly(data)
years = yearly.index.values
yearly['suicides_per_100k'] = get_suicides_per_100k(yearly)

fig = go.Figure()

fig.add_trace(
    go.Scatter(x = years, y = yearly['suicides_per_100k'], mode = 'lines', name = 'Suicides per 100 000 people')
)

fig.update_layout(title_text="Suicides per 100 000 people")

fig.update_xaxes(title_text="years")
fig.update_yaxes(title_text="Suicides per 100 000 people")

fig.show()

**Graph insights**
We can see that the from the 1988 there was a increase in the suicides per capita with peak in 1995. Since then, the suicide rate is decreasing again.

Now let's look at social groups, starting with geneders.

In [314]:
males_yearly = group_yearly(data[data['sex'] == 'Male'])
males_yearly['suicides_per_100k'] = get_suicides_per_100k(males_yearly)

females_yearly = group_yearly(data[data['sex'] == 'Female'])
females_yearly['suicides_per_100k'] = get_suicides_per_100k(females_yearly)


fig = go.Figure()

fig.add_trace(
    go.Scatter(x = years, y = males_yearly['suicides_per_100k'], mode = 'lines', name = 'Males')
)

fig.add_trace(
    go.Scatter(x = years, y = females_yearly['suicides_per_100k'], mode = 'lines', name = 'Females')
)

fig.update_layout(title_text="Suicides per 100 000 people")

fig.update_xaxes(title_text="years")
fig.update_yaxes(title_text="Suicides per 100 000 people")

fig.show()

There seems to be significant difference between the male suicide rate and the female suicide rate. I will investigate this trend further in the following chapters.

### Countries related

In [317]:
countries_data = data.groupby('country').agg(
    {'population': 'sum', 'suicides_no' : 'sum'}
)
countries_data['suicides_per_100k'] = get_suicides_per_100k(countries_data)
countries_data = countries_data.sort_values('suicides_per_100k', ascending=False)

countries_mean = countries_data['suicides_per_100k'].mean()

In [320]:
def plot_countries_rate(df, title):
    fig = px.bar(df,
        y='country',
        x='suicides_per_100k',
        orientation='h', color='country',
        title=title
    )

    fig.add_trace(go.Scatter(
        x=np.repeat(countries_mean, len(largest_rate['country'])),
        y=df['country'],
        line=go.scatter.Line(color="black"),
        hoverinfo='none'
    ))
    
    fig.update_layout(showlegend=False)
    
    return fig

Now let's see some countries related data, graphs with highest and lowest suicide rate. The vertical line is mean from all countries.

In [321]:
plot_countries_rate(
    countries_data['suicides_per_100k'].nlargest(10).sort_values(ascending=False).reset_index(), 
    'Top 10 Countries with highest average of suicide per 100 000 population'
)

In [322]:
plot_countries_rate(
    countries_data['suicides_per_100k'].nsmallest(10).sort_values(ascending=False).reset_index(), 
    '10 Countries with smallest average of suicide per 100 000 population'
)

Interestingly, Dominica and Saint Kitts and Nevis has no suicides whatsoever. This could mean that the data are missing some values, or that the people in these states are really happy with themselves.

In [323]:
countries_data[countries_data['suicides_per_100k'] == 0]

,population,suicides_no,suicides_per_100k
country,,,
Dominica,66400.0,0,0.0
Saint Kitts and Nevis,117300.0,0,0.0


### Data correlation

Let's dig more into the data and try to look, whether is there some correlation between the dataset features.

In [348]:
data.corr()

,year,suicides_no,population,gdp_for_year,gdp_per_capita
year,1.000000,-0.002953,0.011583,0.097220,0.338854
suicides_no,-0.002953,1.000000,0.616105,0.430011,0.062168
population,0.011583,0.616105,1.000000,0.710834,0.083006
gdp_for_year,0.097220,0.430011,0.710834,1.000000,0.304813
gdp_per_capita,0.338854,0.062168,0.083006,0.304813,1.000000
